In [1]:
import asyncio
from uuid import uuid4

In [2]:
from pydantic import BaseModel

In [3]:
from matter_persistence.redis.manager import CacheManager

In [4]:
# example DTO
class Test(BaseModel):
    some_field: int
    some_other_field: list[int]

# test type
t = Test(some_field=0, some_other_field=[_ for _ in range(5)])

In [5]:
# random organisation and internal id
ORGANISATION_ID = uuid4()
INTERNAL_ID = uuid4()

In [6]:
cache_manager = CacheManager(host="localhost", port=6379) # make sure there is a redis instance running!

In [7]:
# check if cache is alive
await cache_manager.is_cache_alive()

True

In [8]:
# save pydantic type to cache
await cache_manager.save_value(ORGANISATION_ID, INTERNAL_ID, t, Test)

In [9]:
# check if pydantic object exists in cache
await cache_manager.cache_record_exists(ORGANISATION_ID, INTERNAL_ID, Test)

1

In [10]:
# find pydantic object in cache
await cache_manager.get_value(ORGANISATION_ID, INTERNAL_ID, Test)

CacheRecordModel(internal_id='d14bf719-bb50-4f0c-b0fa-707d5dd1b0f6', hash_key='156c7600-fdac-4508-8756-f0f9a0a79a8d_Test_6462127acdbd4fed9b665aea70d45c299bcc712f', organization_id=UUID('156c7600-fdac-4508-8756-f0f9a0a79a8d'), value={'some_field': 0, 'some_other_field': [0, 1, 2, 3, 4]}, expiration=None)

In [11]:
# delete pydantic object from cache
await cache_manager.delete_value(ORGANISATION_ID, INTERNAL_ID, Test)

In [12]:
# make sure pydantic object is deleted from cache
await cache_manager.cache_record_exists(ORGANISATION_ID, INTERNAL_ID, Test)

0